In [1]:
import os
import sys

sys.path.append("../../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from src.utils.helper import Config, color_print
from src.utils.load import load_model, load_data, save_checkpoint
from src.models.evaluate import (
    evaluate_model,
    get_sparsity,
    get_similarity,
    get_perplexity,
)
from src.utils.sampling import SamplingDataset
from src.pruning.prune import prune_magnitude
from src.utils.helper import report_to_df, append_nth_row

In [3]:
name = "OSDG"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 16
ratio = 0.4
seed = 44
include_layers = ["attention", "intermediate", "output"]
exclude_layers = None

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-10-22 01:21:57


In [5]:
config = Config(name, device)
num_labels = config.config["num_labels"]
model = load_model(config)

Loading the model.

{

'architectures'

: 

'bert'

...

'tokenizer_name'

: 

'sadickam/sdg-classification-bert'

}

The model sadickam/sdg-classification-bert is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    config,
    batch_size=batch_size,
    num_workers=num_workers,
    do_cache=True,
)

Loading cached dataset OSDG.

train.pkl is loaded from cache.

valid.pkl is loaded from cache.

test.pkl is loaded from cache.

The dataset OSDG is loaded

...

,
 

'path'

: 

'albertmartinez/OSDG'

}

In [7]:
# print("Evaluate the original model")
# result = evaluate_model(model, config, test_dataloader)

In [8]:
config.init_seed()
all_samples = SamplingDataset(
    train_dataloader,
    config,
    200,
    num_samples,
    False,
    4,
    resample=False,
)

In [9]:
result_list = []

module = copy.deepcopy(model)
prune_magnitude(
    module,
    sparsity_ratio=ratio,
    include_layers=include_layers,
    exclude_layers=exclude_layers,
)
print("Evaluate the pruned model")
result = evaluate_model(model, config, test_dataloader, verbose=True)
result_list.append(result)

Evaluate the pruned model

Evaluating the model:   0%|                                                                                | 0…

In [10]:
for concern in range(num_labels):
    config.init_seed()
    get_similarity(model, module, valid_dataloader, concern, num_samples, config)

Exception ignored in: 

<function _MultiProcessingDataLoaderIter.__del__ at 0x7fed86ea2840>

Traceback (most recent call last):


  File "/home/jieungkim/.cache/pypoetry/virtualenvs/decomposetransformer-UESb9BbT-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__


self._shutdown_workers()

...

CCA coefficients mean non-concern: 0.7394076944518252

Linear CKA concern: 0.9193913393854787

Linear CKA non-concern: 0.9145778528466064

Kernel CKA concern: 0.9076701604553586

Kernel CKA non-concern: 0.9151140221336947

In [11]:
get_sparsity(module)
print("original model's perplexity")
get_perplexity(model, valid_dataloader, config)
print("pruned model's perplexity")
get_perplexity(module, valid_dataloader, config)

0.39679545888171447

{'bert.encoder.layer.0.attention.self.query.weight': 0.3999989827473958, 'bert.encoder.layer.0.attention.self.query.bias': 0.0, 'bert.encoder.layer.0.attention.self.key.weight': 0.3999989827473958, 'bert.encoder.layer.0.attention.self.key.bias': 0.0, 'bert.encoder.layer.0.attention.self.value.weight': 0.3999989827473958, 'bert.encoder.layer.0.attention.self.value.bias': 0.0, 'bert.encoder.layer.0.attention.output.dense.weight': 0.3999989827473958, 'bert.encoder.layer.0.attention.output.dense.bias': 0.0, 'bert.encoder.layer.0.intermediate.dense.weight': 0.39999983045789933, 'bert.encoder.layer.0.intermediate.dense.bias': 0.0, 'bert.encoder.layer.0.output.dense.weight': 0.39999983045789933, 'bert.encoder.layer.0.output.dense.bias': 0.0, 'bert.encoder.layer.1.attention.self.query.weight': 0.3999989827473958, 'bert.encoder.layer.1.attention.self.query.bias': 0.0, 'bert.encoder.layer.1.attention.self.key.weight': 0.3999989827473958, 'bert.encoder.layer.1.attention.self.key.bias': 0.0, 'bert

original model's perplexity

2.445301055908203

pruned model's perplexity

2.4151687622070312

2.4151687622070312

In [12]:
df_list = [report_to_df(df) for df in result_list]
csv_name = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
new_df = df_list[0]
new_df.to_csv(f"results/{csv_name}.csv", index=False)
print(csv_name)
new_df

2024-10-22_01-29-08

,class,precision,recall,f1-score,support
0,0,0.7697,0.6625,0.7121,797
1,1,0.8450,0.7174,0.7760,775
2,2,0.8809,0.8742,0.8775,795
3,3,0.8733,0.8324,0.8524,1110
4,4,0.8635,0.8032,0.8322,1260
5,5,0.8810,0.6882,0.7728,882
6,6,0.8508,0.8011,0.8252,940
7,7,0.4898,0.6089,0.5429,473
8,8,0.6553,0.8539,0.7416,746
9,9,0.6151,0.7329,0.6689,689
